import essential packages

In [1]:
from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies
from pyscenic.prune import prune2df, df2regulons
from dask.diagnostics import ProgressBar
from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
from pyscenic.aucell import aucell


import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import pybedtools
import gffutils
import shutil
import os, glob
import ctxcore
import pickle
import time
import re

D:\Anaconda\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
D:\Anaconda\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
D:\Anaconda\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


ModuleNotFoundError: No module named 'pybedtools'

import essential files

In [ ]:
#data folder
DATA_FOLDER="GSM4878210_L+H_E1_QC"###your folder
# GRNBoost input
SC_EXP_FNAME = os.path.join(DATA_FOLDER, "GSE115968.csv")#inpput gene expression matrix
MM_TFS_FNAME = os.path.join(DATA_FOLDER, 'mm_tfs.txt')#tf list

#NIRA INPUT
TSS_ANNOTATION = os.path.join(DATA_FOLDER, "mm10TSS(/hg38TSS).csv")#prebuilt database which can be downloaded from figshare'
Hits = os.path.join(DATA_FOLDER, hitsFolder)#prebuilt database which can be downloaded from figshare' which instore all the occurance for each motifs
ATAC_PEAK = os.path.join(DATA_FOLDER, "ATACmm10.bed")#input ATAC peaks
#FILE SAVED
MODULES_FNAME = os.path.join(DATA_FOLDER, "modules.p")




grnboost

In [ ]:
tf_names = load_tf_names(MM_TFS_FNAME)
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
tic = time.time()
ex_matrix = pd.read_csv(SC_EXP_FNAME, sep=',', header=0, index_col=0)#cell in col; gene in row
#IF NEEDED:ex_matrix = ex_matrix.T
adjancencies = grnboost2(expression_data=ex_matrix, tf_names=tf_names, verbose=True)
modules = list(modules_from_adjacencies(adjancencies,ex_matrix,rho_mask_dropouts=True))
with open(MODULES_FNAME, 'wb') as f:
    pickle.dump(modules, f)
tok = time.time() 
print(tok-tic)

In [ ]:
for i in range(1,len(modules)):
    if modules[i].name == modules[0].name:
        print(i)
        break
modules = modules[0:i-1]

-------------      NIRA functions

main function:

In [ ]:
def NIRAregulon(modules,TSSD,ATAC):

######center the atac
    ATAC = np.mat(ATAC)
    ATAC[:,1] = (ATAC[:,1]+ATAC[:,2])/2-200
    ATAC[:,2] = (ATAC[:,1]+ATAC[:,2])/2+200
    ATAC=pd.DataFrame(ATAC)
    ATAC[[1]]=ATAC[[1]].astype(int)
    ATAC[[2]]=ATAC[[2]].astype(int)
    ATACbed = BedTool.from_dataframe(ATAC)

    mylist = []
    dict_reg={}
    for m in modules:
        TFname = m.name.replace("Regulon for ","").upper()
        tss_all = TSSD[TSSD['external_gene_name'].isin(m.genes)]
        tss_d = tss_all[["chromosome_name","TSS-D","TSS+D","external_gene_name","transcription_start_site"]]
        if os.path.exists(fileset+'//'+TFname+".bed") == True:
            #calculate the backgroud based on the data population
            hits = pd.read_table("hits//"+TFname+".bed")
            hits = BedTool.from_dataframe(hits)
            hitsperpeak = ATACbed.intersect(hits,wa=True,wb=True)
            hitsperpeak = hitsperpeak.to_dataframe(header=None)
            mathits =np.mat(hitsperpeak)
            #normalization
            mathits[:,6]=(mathits[:,6]-mathits[:,6].min())/(mathits[:,6].max()-mathits[:,6].min())
            hitsperpeak = pd.DataFrame(mathits)
            hitsperpeak = BedTool.from_dataframe(hitsperpeak)
            hitsperpeak = hitsperpeak.sort()
            hitsperpeakbed = hitsperpeak.merge(c="7", o="sum")
            hitsperpeak = hitsperpeakbed.to_dataframe(header=None)
            #calculate the population
            mean = np.mean(hitsperpeak[["name"]])
            var = np.std(hitsperpeak[["name"]])
            #calculate the confidence interval
            cinter = stats.norm.interval(0.99, loc=mean, scale=var/math.sqrt(1000))

            #for given regulons:
            tss_all = TSSD[TSSD['external_gene_name'].isin(m.genes)]
            tss_d = tss_all[["chromosome_name","TSS-D","TSS+D","external_gene_name","transcription_start_site"]]
            x = BedTool.from_dataframe(tss_d)
            peakswithin = x.intersect(hitsperpeakbed,wa=True,wb=True)
            hits0perpeak = peakswithin.to_dataframe(header=None)
            y = hits0perpeak[["strand","thickStart","thickEnd","itemRgb"]]
            y = BedTool.from_dataframe(y)
            y = y.sort()
            y = y.merge(c="4", o="mean")
            y = y.to_dataframe(header=None)
            nhits0 = np.mean(y[["name"]])[0]
            if nhits0>=cinter[1]:
                peaksperhits0within = peakswithin.sort()
                ssdd = peaksperhits0within.merge(c="4,5,9", o="distinct,mean,sum")
                Mat = calculate(ssdd)
                Mat.columns = ["GENE","SCORE"]
                #m_1 = file.shape[0]
                #file_1 = Mat.loc[file["SCORE"]>#####]
                #m_2 = file_1.shape[0]
                #if m_2 >20:
                innerdict = Mat.set_index('GENE').T.to_dict('list')
                #regulons = Mat["GENE"].values.to_list()
                scores = Mat['SCORE'].values.to_list()
                mylist.append(scores)
                dict_reg[TFname]=innerdict
    
    myonelist =[i for p in mylist for i in p]
    thres = threshentroy(myonelist)[0]
    filter_reg={}
    for i in len(dict_reg):
        REGname = dict_reg[i]
        filter_Reg ={k for k,v in dict_reg[REGname].items() if v[0]>=thres}
        #filter_Reg = dict(filter(lambda item:item>=thres,regulons.item()))
        filter_reg[REGname]=filter_Reg
        
    return filter_reg

side functions:

In [ ]:
def calculate(ssdd):
    file = ssdd.to_dataframe(header=None,names=["Chr","start","end","genename","genetss","score"])
    tt = file["genename"].str.split(",",expand=True)
    if tt.shape[1] == 1:
        x = file [["start"]]
        x = np.mat(x)
        y = file[["end"]]
        y = np.mat(y)
        z=file[["genetss"]]
        z=np.mat(z)
        d = abs(z-(x+y)/2)
        d = d.astype(float)
        d = np.exp(-d/5000)
        s=file[["score"]]
        s=np.mat(s)
        bc_ = d*s.T
        diagonal = np.diag_indices(len(d))
        bc_ = bc_[diagonal]
        gene=file[["genename"]]
        gene=np.mat(gene)
        bc = np.append(gene,bc_.T,axis=1)
        bc= pd.DataFrame(bc)
        bc.columns=["gene","score"]
        
        l = bc.groupby('gene').agg({'score':np.sum})
        id = l.index 
        id = id.tolist()
        SVmatrix = l[["score"]]
        SVmatrix=SVmatrix.values.tolist()
        SVmatrix =np.mat(SVmatrix)
        t = pd.DataFrame(m.weights)
        t.index = m.genes
        t.columns = ["IM"]
        IM  = np.mat(t.loc[id][["IM"]])
        Wmatrix = IM*SVmatrix.T
        diagonal = np.diag_indices(len(IM))
        Wmatrix = Wmatrix[diagonal]
        Wmatrix = pd.DataFrame(Wmatrix.T)
        id = pd.DataFrame(id)
        Wdf = pd.concat([id,Wmatrix],axis=1)
    else: 
       
        r = tt[tt[1].notnull()== True].index
        v = pd.DataFrame(columns = ["Chr","start","end","genename","genetss","score"])
        for j in range(len(r)):
            s = r[j] 
            n = len(file.loc[s]["genename"].split(","))
            for i in range(n):
                xx = file.loc[s]
                xx["genename"] = file.loc[s]["genename"].split(",")[i]
                v.loc[2*j+i]=xx
        file = file.append(v, ignore_index=True)
        file = file.drop(r)

        x = file [["start"]]
        x = np.mat(x)
        y = file[["end"]]
        y = np.mat(y)
        z=file[["genetss"]]
        z=np.mat(z)
        d = abs(z-(x+y)/2)
        d = d.astype(float)
        d = np.exp(-d/5000)
        s=file[["score"]]
        s=np.mat(s)
        bc_ = d*s.T
        diagonal = np.diag_indices(len(d))
        bc_ = bc_[diagonal]
        gene=file[["genename"]]
        gene=np.mat(gene)
        bc = np.append(gene,bc_.T,axis=1)
        bc= pd.DataFrame(bc)
        bc.columns=["gene","score"]
        l = bc.groupby('gene').agg({'score':np.sum})
        id = l.index 
        id = id.tolist()
        SVmatrix = l[["score"]]
        SVmatrix=SVmatrix.values.tolist()
        SVmatrix =np.mat(SVmatrix)
        t = pd.DataFrame(m.weights)
        t.index = m.genes
        t.columns = ["IM"]
        IM  = np.mat(t.loc[id][["IM"]])
        Wmatrix = IM*SVmatrix.T
        diagonal = np.diag_indices(len(IM))
        Wmatrix = Wmatrix[diagonal]
        Wmatrix = pd.DataFrame(Wmatrix.T)
        id = pd.DataFrame(id)
        Wdf = pd.concat([id,Wmatrix],axis=1)
    return Wdf

In [ ]:
def threshentroy(x):
    n0 = len(x)
    maxv = max(x)##### modification could be used
    ######## probhist
    counts0 = np.histogram(x,bins=int(maxv))
    newcounts =list(filter(lambda x:x>10, counts0[0]))
    counts = np.mat(newcounts)
    counts = counts/n0
    counts = counts.tolist()
    counts = counts[0]
   
    n = len(counts)
    #####sumprob for each cut
    eee = np.zeros(n)
    
    for k in range(n):
        amat = np.zeros(n)
        asum = np.sum(counts[:k+1])
        for j in range(k+1):
            if counts[j]==0 or asum ==0:
                amat[j]=0
            else:
                amat[j] = -(counts[j]/asum)*(np.log10((counts[j]/asum)))
        for i in range(k+1,n):
            if counts[i]==0 or asum==1:
                amat[i]=0
            else:
                amat[i] =-(counts[i]/(1-asum))*(np.log10(counts[i]/(1-asum)))
        
        f1= np.sum(amat[:k+1])
        f2= np.sum(amat[k+1:])
        eee[k]=f1+f2

    threshold = np.where(eee==np.max(eee))
    thres = threshold[0][0]
    return counts0[1][thres],threshold

In [ ]:
TSS_ANNOTATION = pd.read_csv(SC_EXP_FNAME, sep=',', header=0)
ATAC_PEAK = pd.read_table(ATAC_PEAK)

In [ ]:
regulons = NIRAregulon(modules,TSS_ANNOTATION,ATAC_PEAK)

------------- AUCELL for each regulon input and BOOTSTRAP

rank the expression data for each cell:

In [ ]:
from pyscenic.genesig import GeneSignature
from pyscenic.aucell import create_rankings, enrichment

In [ ]:
genesets = ex_matrix.T.index
nreg =len(filter_reg)
ngene = len(genesets)
m,n=np.shape(ex_matrix)
nTest=10000
#original AUC
M =np.zeros([0,0])
M=pd.DataFrame(M)
#BOOTSTRAP AUC
N =np.zeros([0,0])
N=pd.DataFrame(N)
#binary AUC
R = np.zeros(shape=(m,nTest+1))

In [ ]:
rnk_mtx = create_rankings(ex_matrix)

In [ ]:
start=time.time()
for i in filter_reg:
    reg = filter_reg[i]
    s = len(reg)
    gs0 = GeneSignature(i,[line.strip() for idx, line in enumerate(reg)])
    aucs0 = enrichment(rnk_mtx, gs)
    vv = np.mat(aucs0[["AUC"]])
    R[:,0] = vv.T
    M[[i]] = pd.DataFrame(vv)
    for j in range(0,nTest):
        rs = np.random.choice(genesets,s,False)
        gs = GeneSignature(i,[line.strip() for idx, line in enumerate(rs)])
        aucs = enrichment(rnk_mtx, gs)
        R[:,j+1] = np.mat(aucs[["AUC"]]).T

    Rm = R[:,1:].T- R[:,0].T
    Rm=Rm.T
    Pm = pd.DataFrame(Rm)
    pv = ((Pm >= 0).astype(int).sum(axis=1))/nTest
    pv = pd.DataFrame(pv)
    N[[i]]= pv
end=time.time()
print(end-start)